# Algorytm k najbliższych sąsiadów

W ćwiczeniu wykorzystywany jest zbiór danych `donors.csv`, opracowany przez organizację weteranów, która zabiga o datki, kierując kampanie listowe do aktualnych i potencjalnych darczyńców z bazy danych. Zbiór danych zawiera informacje o cechach charakterystycznych klientów oraz o tym, czy zareagowali na kampanię testową. Zadaniem jest pomoc organizacji w przewidzeniu, którzy darczyńcy odpowiedzą na kampanię listową w oparciu o dane demograficzne, wcześniejszą historię datków oraz reakcję na poprzednie kampanie listowe.

1. Zaimportuj dane i wyświetl ich podgląd (funkcje `read_csv` oraz `glimpse`. Pierwsze 12 zmiennych jest typu `numeric`, pozostałe 10 jest typu `factor`). Cecha klasy nosi nazwę `respondentMailing`.

In [ ]:
if (!require(tidyverse)) install.packages("tidyverse")
if (!require(class)) install.packages("class")
if (!require(performanceEstimation)) install.packages("performanceEstimation")

library(tidyverse)
library(class)
library(performanceEstimation)

In [ ]:
df <- read.csv("donors.csv")

head(df)

,age,numberChildren,incomeRating,wealthRating,mailOrderPurchases,totalGivingAmount,numberGifts,smallestGiftAmount,largestGiftAmount,averageGiftAmount,⋯,inHouseDonor,plannedGivingDonor,sweepstakesDonor,P3Donor,state,urbanicity,socioEconomicStatus,isHomeowner,gender,respondedMailing
,<int>,<int>,<int>,<int>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,⋯,<lgl>,<lgl>,<lgl>,<lgl>,<chr>,<chr>,<chr>,<lgl>,<chr>,<lgl>
1,60,NA,NA,NA,0,240,31,5,12,7.741935,⋯,FALSE,FALSE,FALSE,FALSE,IL,town,average,NA,female,FALSE
2,46,1,6,9,16,47,3,10,25,15.666667,⋯,FALSE,FALSE,FALSE,FALSE,CA,suburb,highest,TRUE,male,FALSE
3,NA,NA,3,1,2,202,27,2,16,7.481481,⋯,FALSE,FALSE,FALSE,FALSE,NC,rural,average,NA,male,FALSE
4,70,NA,1,4,2,109,16,2,11,6.812500,⋯,FALSE,FALSE,FALSE,FALSE,CA,rural,average,NA,female,FALSE
5,78,1,3,2,60,254,37,3,15,6.864865,⋯,TRUE,FALSE,FALSE,TRUE,FL,suburb,average,TRUE,female,FALSE
6,NA,NA,NA,NA,0,51,4,10,16,12.750000,⋯,FALSE,FALSE,FALSE,FALSE,AL,town,average,NA,NA,FALSE


In [ ]:
dim(df)

[1] 95412    22

In [ ]:
df <- df %>%
  mutate(
         age = as.integer(age),
         numberChildren = as.integer(numberChildren),
         incomeRating = as.integer(incomeRating),
         wealthRating = as.integer(wealthRating),
         mailOrderPurchases = as.integer(mailOrderPurchases),
         totalGivingAmount = as.integer(totalGivingAmount),
         numberGifts = as.integer(numberGifts),
         smallestGiftAmount = as.integer(smallestGiftAmount),
         largestGiftAmount = as.integer(largestGiftAmount),
         averageGiftAmount = as.double(averageGiftAmount),
         yearsSinceFirstDonation = as.integer(yearsSinceFirstDonation),
         monthsSinceLastDonation = as.integer(monthsSinceLastDonation),
         inHouseDonor = as.factor(inHouseDonor),
         plannedGivingDonor = as.factor(plannedGivingDonor),
         sweepstakesDonor = as.factor(sweepstakesDonor),
         P3Donor = as.factor(P3Donor),
         state = as.factor(state),
         urbanicity = as.factor(urbanicity),
         socioEconomicStatus = as.factor(socioEconomicStatus),
         isHomeowner = as.factor(isHomeowner),
         gender = as.factor(gender),
         respondedMailing = as.factor(respondedMailing))

glimpse(df)

Rows: 95,412
Columns: 22
$ age                     <int> 60, 46, NA, 70, 78, NA, 38, NA, NA, 65, NA, 75…
$ numberChildren          <int> NA, 1, NA, NA, 1, NA, 1, NA, NA, NA, NA, NA, 2…
$ incomeRating            <int> NA, 6, 3, 1, 3, NA, 4, 2, 3, NA, 2, 1, 4, NA, …
$ wealthRating            <int> NA, 9, 1, 4, 2, NA, 6, 9, 2, NA, 0, 5, 2, NA, …
$ mailOrderPurchases      <int> 0, 16, 2, 2, 60, 0, 0, 1, 0, 0, 0, 3, 16, 0, 1…
$ totalGivingAmount       <int> 240, 47, 202, 109, 254, 51, 107, 31, 199, 28, …
$ numberGifts             <int> 31, 3, 27, 16, 37, 4, 14, 5, 11, 3, 1, 2, 9, 1…
$ smallestGiftAmount      <int> 5, 10, 2, 2, 3, 10, 3, 5, 10, 3, 20, 10, 4, 5,…
$ largestGiftAmount       <int> 12, 25, 16, 11, 15, 16, 12, 11, 22, 15, 20, 15…
$ averageGiftAmount       <dbl> 7.741935, 15.666667, 7.481481, 6.812500, 6.864…
$ yearsSinceFirstDonation <int> 8, 3, 7, 10, 11, 3, 10, 3, 9, 3, 1, 1, 8, 5, 4…
$ monthsSinceLastDonation <int> 14, 14, 14, 14, 13, 20, 22, 18, 19, 22, 12, 14…
$ inHouseDonor 

2. Ogranicz predyktory wyłącznie do cech liczbowych ze zbioru danych (funkcja `select`). Wyświetl podsumowanie statystyczne dla nowego zbioru danych i sprawdź dla których zmiennych występują braki danych.

In [ ]:
df_numeric <- df %>%
    select(where(is.numeric))

df_numeric["respondedMailing"] <- df["respondedMailing"]

In [ ]:
numberofna <- df_numeric %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

In [ ]:
round(numberofna / nrow(df_numeric) * 100, 2)

[1] 91.86

3. Uzupełnij brakujące dane: dla zmiennej `age` użyj imputacji średniej, dla zmiennej `numberChildren` - imputacji mediany. W przypadku cech `incomeRating` oraz `wealthRating` problem brakujących danych rozwiąż wykluczając te wystąpienia ze zbioru danych. Ponadto dla zmiennej `wealthRating` (miara ogólnego majątku darczyńcy w skali od 1 do 9) należy również wykluczyć wystąpienia o wartości 0.

In [ ]:
df_numeric["age"][is.na(df_numeric["age"])] <- as.integer(mean(df_numeric$age, na.rm = TRUE))
df_numeric["numberChildren"][is.na(df_numeric["numberChildren"])] <- median(df_numeric$numberChildren, na.rm = TRUE)

df_numeric <- df_numeric %>%
    filter(!is.na(incomeRating) &
           !is.na(wealthRating) &
           wealthRating > 0
    )

In [ ]:
df_numeric %>%
  filter(if_any(everything(), is.na)) %>%
  nrow()

[1] 0

In [ ]:
dim(df_numeric)

[1] 48267    13

4. Utwórz funkcję normalizacji min-max i zapisz ją jako `normalize`. Następnie wykorzystaj ją do znormalizowania wartości cech (ustandaryzowania skali do zakresu od 0 do 1).

In [ ]:
min_max <- function(x) {
    return ( (x - min(x)) / (max(x) - min(x)) )
}

df_numeric_norm <- data.frame(sapply(df_numeric, function(x) if (is.numeric(x)) min_max(x) else x))
df_numeric_norm["respondedMailing"] <- as.factor(df_numeric_norm["respondedMailing"] == 2)

In [ ]:
nrow(df_numeric_norm)

[1] 48267

5. Podziel dane na zbiory treningowy i testowy w stosunku 75:25, przekształcając uprzednio dane w ramkę danych. Porównaj rozkłady klas dla oryginalnego zbioru danych oraz zbiorów treningowego i testowego.

In [ ]:
set.seed(2024)
split <- sample(1:nrow(df_numeric_norm),
                      size = 0.75 * nrow(df_numeric_norm))

X_train <- df_numeric_norm[split, ]
X_test <- df_numeric_norm[-split, ]

In [ ]:
names(X_train)

[1] "age"                     "numberChildren"         
 [3] "incomeRating"            "wealthRating"           
 [5] "mailOrderPurchases"      "totalGivingAmount"      
 [7] "numberGifts"             "smallestGiftAmount"     
 [9] "largestGiftAmount"       "averageGiftAmount"      
[11] "yearsSinceFirstDonation" "monthsSinceLastDonation"
[13] "respondedMailing"

In [ ]:
print_class_dist <- function(data, name) {
    cat(glue::glue("{name} dataset class distribution"))
    data %>%
    group_by(respondedMailing) %>%
    count(name = "count") %>%
    mutate(perc_dist = count/nrow(data))
}

In [ ]:
print_class_dist(df, "Whole")

Whole dataset class distribution

respondedMailing,count,perc_dist
<fct>,<int>,<dbl>
FALSE,90569,0.94924119
TRUE,4843,0.05075881


In [ ]:
print_class_dist(X_train, "Training")

Training dataset class distribution

respondedMailing,count,perc_dist
<fct>,<int>,<dbl>
FALSE,34325,0.94820442
TRUE,1875,0.05179558


In [ ]:
print_class_dist(X_test, "Test")

Test dataset class distribution

respondedMailing,count,perc_dist
<fct>,<int>,<dbl>
FALSE,11445,0.94845446
TRUE,622,0.05154554


6. Za pomocą funkcji `smote` z pakietu `performanceEstimation` zrównoważ dane treningowe.

In [ ]:
smote_data <- smote(respondedMailing ~ ., data = X_train, perc.over = 9, perc.under = 1.1, k = 5)
print_class_dist(smote_data, "Smoted")

nrow(smote_data)
nrow(X_train)

Smoted dataset class distribution

respondedMailing,count,perc_dist
<fct>,<int>,<dbl>
FALSE,18562,0.4974807
TRUE,18750,0.5025193


[1] 37312

[1] 36200

In [ ]:
X_train <- smote_data

7. Przenieś etykiety klas do osobnych zbiorów danych. Z pomocą polecenia `pull()` z pakietu `tidyverse` utwórz nowe wektory z etykietami cechy klasy (`respondedMailing`) i przekształć je do typu `factor`.

In [ ]:
train_labels <- X_train %>%
  pull(respondedMailing) %>%
  as.factor()

test_labels <- X_test %>%
  pull(respondedMailing) %>%
  as.factor()

8. Przekształć zbiory danych treningowych i testowych do postaci ramek danych bez etykiet klas (`respondedMailing`).

In [ ]:
train_features <- X_train %>%
  select(-respondedMailing)

test_features <- X_test %>%
  select(-respondedMailing)

9. Budowanie modelu: Wykorzystaj funkcję `knn()` z pakietu `class` do oznaczenia przykładów testowych na podstawie danych treningowych. `k` ustaw wartość `5`.

In [ ]:
set.seed(123)
predictions <- knn(train = train_features, test = test_features, cl = train_labels, k = 5)

10. Wyświetl pierwsze 6 predykcji.

In [ ]:
head(predictions)

[1] FALSE TRUE  FALSE FALSE TRUE  FALSE
Levels: FALSE TRUE

11. Utwórz macierz pomyłek i na jej podstawie oblicz dokładność predykcji.

In [ ]:
conf_matrix <- table(Actual = test_labels, Predicted = predictions)
print(conf_matrix)

accuracy <- sum(diag(conf_matrix)) / sum(conf_matrix)
cat("Accuracy:", round(accuracy * 100, 2), "%\n")

       Predicted
Actual  FALSE TRUE
  FALSE  8034 3411
  TRUE    426  196
Accuracy: 68.2 %


12. Zmodyfikuj kod tak, aby użyć zmiennych kategorialnych w celu poprawy dokładności modelu.

In [ ]:
# Szukamy missing values
numberofna <- df %>%
    filter(if_any(everything(), is.na)) %>%
    nrow()
print(paste("Percentage of rows with NA:", round(numberofna / nrow(df) * 100, 2), "%"))

[1] "Percentage of rows with NA: 92.71 %"


In [ ]:
# Uzupełnienie brakujących wartości
df["age"][is.na(df["age"])] <- as.integer(mean(df$age, na.rm = TRUE))
df["numberChildren"][is.na(df["numberChildren"])] <- median(df$numberChildren, na.rm = TRUE)

In [ ]:
# Filtrujemy brakujące wartości w wealthRating i incomeRating
df <- df %>%
    filter(!is.na(incomeRating) &
           !is.na(wealthRating) &
           wealthRating > 0)

In [ ]:
# Sprawdzamy ponownie brakujące wartości
df %>%
    filter(if_any(everything(), is.na)) %>%
    nrow()

[1] 13702

In [ ]:
# Usuwamy zmienną 'state' która ma zbyt dużo kategorii
df <- df[-20]

In [ ]:
# Usuwamy brakujące wartości
df <- df %>%
    filter(!if_any(everything(), is.na))

In [ ]:
# Tworzymy znormalizowany dataset z numerycznymi i kategorycznymi zmiennymi
df_norm <- cbind(data.frame(sapply(df[1:12], function(x) min_max(x))), model.matrix(~ . - 1, data = df[13:20]), df[21])

In [ ]:
# Wyświetlamy wymiary znormalizowanego datasetu
print("Dimensions of normalized dataset:")
dim(df_norm)

[1] "Dimensions of normalized dataset:"


[1] 46168    82

In [ ]:
# Sprawdzamy brakujące wartości
print("Any NA values remaining?")
any(is.na(df_norm))

[1] "Any NA values remaining?"


[1] FALSE

In [ ]:
# Podział na zbiór treningowy i testowy
set.seed(123)
split <- sample(1:nrow(df_norm), size = 0.75 * nrow(df_norm))
X_train <- df_norm[split, ]
X_test <- df_norm[-split, ]

In [ ]:
# Wyświetlamy rozkład klas
print_class_dist <- function(data, name) {
    cat("\n", name, "dataset class distribution:\n")
    prop.table(table(data$respondedMailing)) %>%
    round(4) %>%
    print()
}

print_class_dist(df_norm, "Whole")
print_class_dist(X_train, "Training")
print_class_dist(X_test, "Test")


 Whole dataset class distribution:

 FALSE   TRUE 
0.9483 0.0517 

 Training dataset class distribution:

 FALSE   TRUE 
0.9483 0.0517 

 Test dataset class distribution:

 FALSE   TRUE 
0.9482 0.0518 


In [ ]:
print_class_dist(df_norm, "Whole")
print_class_dist(X_train, "Training")
print_class_dist(X_test, "Test")


 Whole dataset class distribution:

 FALSE   TRUE 
0.9483 0.0517 

 Training dataset class distribution:

 FALSE   TRUE 
0.9483 0.0517 

 Test dataset class distribution:

 FALSE   TRUE 
0.9482 0.0518 


In [ ]:
# Używamy SMOTE do zbalansowania danych treningowych
smote_data <- smote(respondedMailing ~ ., data = X_train, perc.over = 9, perc.under = 1.1, k = 5)

print_class_dist(smote_data, "SMOTE-balanced")
print(paste("Oryginalny training set size:", nrow(X_train)))
print(paste("SMOTE-balanced training set size:", nrow(smote_data)))


 SMOTE-balanced dataset class distribution:

 FALSE   TRUE 
0.4975 0.5025 
[1] "Oryginalny training set size: 34626"
[1] "SMOTE-balanced training set size: 35640"


In [ ]:
# Zmieniamy zbiór treningowy
X_train <- smote_data

In [ ]:
# Podział na feature'y i labels
train_labels <- X_train$respondedMailing
test_labels <- X_test$respondedMailing

In [ ]:
# Usuwamy zmienną 'respondedMailing'
X_train <- X_train %>% select(-respondedMailing)
X_test <- X_test %>% select(-respondedMailing)

In [ ]:
# Uzyskujemy predykcje
set.seed(123)
predictions <- knn(train = X_train, test = X_test, cl = train_labels, k = 5)

In [ ]:
# Tworzymy macierz pomyłek
conf_matrix <- table(Actual = test_labels, Predicted = predictions)
print("Confusion Matrix:")
print(conf_matrix)

[1] "Confusion Matrix:"
       Predicted
Actual  FALSE TRUE
  FALSE  9507 1437
  TRUE    526   72


In [ ]:
# Obliczamy dokładność
accuracy <- sum(diag(conf_matrix)) / sum(conf_matrix)
cat("\nAccuracy:", round(accuracy * 100, 2), "%\n")

# Obliczamy precyzję, recall i f1 score
precision <- conf_matrix[2,2] / sum(conf_matrix[,2])
recall <- conf_matrix[2,2] / sum(conf_matrix[2,])
f1_score <- 2 * (precision * recall) / (precision + recall)

cat("\nInne metryki:\n")
cat("Precision:", round(precision * 100, 2), "%\n")
cat("Recall:", round(recall * 100, 2), "%\n")
cat("F1 Score:", round(f1_score * 100, 2), "%\n")


Accuracy: 82.99 %

Inne metryki:
Precision: 4.77 %
Recall: 12.04 %
F1 Score: 6.83 %
